<h1> Scaling up ML using Cloud ML Engine </h1>

In this notebook, we take a previously developed TensorFlow model to predict taxifare rides and package it up so that it can be run in Cloud MLE. For now, we'll run this on a small dataset. The model that was developed is rather simplistic, and therefore, the accuracy of the model is not great either.  However, this notebook illustrates *how* to package up a TensorFlow model to run it within Cloud ML. 

Later in the course, we will look at ways to make a more effective machine learning model.

<h2> Environment variables for project and bucket </h2>

Note that:
<ol>
<li> Your project id is the *unique* string that identifies your project (not the project name). You can find this from the GCP Console dashboard's Home page.  My dashboard reads:  <b>Project ID:</b> cloud-training-demos </li>
<li> Cloud training often involves saving and restoring model files. If you don't have a bucket already, I suggest that you create one from the GCP console (because it will dynamically check whether the bucket name you want is available). A common pattern is to prefix the bucket name by the project id, so that it is unique. Also, for cost reasons, you might want to use a single region bucket. </li>
</ol>
<b>Change the cell below</b> to reflect your Project ID and bucket name.


In [3]:
import os
PROJECT = 'qwiklabs-gcp-04527c06d4f6cef3' # REPLACE WITH YOUR PROJECT ID
BUCKET = 'qwiklabs-gcp-04527c06d4f6cef3-bucket' # REPLACE WITH YOUR BUCKET NAME
REGION = 'us-central1' # REPLACE WITH YOUR BUCKET REGION e.g. us-central1

In [4]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.7'  # Tensorflow version

In [5]:
%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].
Updated property [core/project].
Updated property [compute/region].


Allow the Cloud ML Engine service account to read/write to the bucket containing training data.

In [6]:
%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print response['serviceAccount']")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

Authorizing the Cloud ML Service account service-731166470827@cloud-ml.google.com.iam.gserviceaccount.com to access files in qwiklabs-gcp-04527c06d4f6cef3-bucket
Authorizing the Cloud ML Service account service-731166470827@cloud-ml.google.com.iam.gserviceaccount.com to access files in qwiklabs-gcp-04527c06d4f6cef3-bucket


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   233    0   233    0     0    330      0 --:--:-- --:--:-- --:--:--   330
No changes to gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/
No changes to gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/lab3a_180630_042957/835dec6c6e6de069147b1d16dc563e55933d5cf771af5f37c09e7ab7fca15b5c/volume_estimator-0.0.1.tar.gz
Updated ACL on gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/
Updated ACL on gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/smallinput/
Updated ACL on gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/smallinput/volume_estimator_trained/
Updated ACL on gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/smallinput/volume_estimator_trained/eval/
Updated ACL on gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/smallinput/volume_estimator_trained/checkpoint
Updated ACL on gs://qwikla

<h2> Packaging up the code </h2>

Take your code and put into a standard Python package structure.  <a href="taxifare/trainer/model.py">model.py</a> and <a href="taxifare/trainer/task.py">task.py</a> contain the Tensorflow code from earlier (explore the <a href="taxifare/trainer/">directory structure</a>).

In [ ]:
!find volume_estimator

In [ ]:
!cat volume_estimator/trainer/model.py

<h2> Find absolute paths to your data </h2>

Note the absolute paths below. /content is mapped in Datalab to where the home icon takes you

In [7]:
%bash
echo $PWD
rm -rf $PWD/volume_estimator_trained

/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow
/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow


<h2> Running the Python module from the command-line </h2>

In [15]:
%bash
rm -rf volume_estimator.tar.gz volume_estimator_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/volume_estimator
python -m trainer.task \
   --output_dir=${PWD}/volume_estimator_trained \
   --train_steps=1000 --job-dir=./tmp \
   --train_data_size=1000 \
   --eval_data_size=1000

/usr/local/envs/py2env/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_train_distribute': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f88f90937d0>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/content/datalab/notebooks/training-data-analyst/courses/machine_learning/deepdive/03_tensorflow/vo

In [12]:
%bash
ls $PWD/volume_estimator_trained/export/exporter/

1530335166
1530335166


In [13]:
%writefile ./test.json
{"r": 1.0,"h": 1.0}

Overwriting ./test.json
Overwriting ./test.json


In [14]:
%bash
model_dir=$(ls ${PWD}/volume_estimator_trained/export/exporter)
gcloud ml-engine local predict \
    --model-dir=${PWD}/volume_estimator_trained/export/exporter/${model_dir} \
    --json-instances=./test.json

PREDICTIONS
[2.5062191486358643]
PREDICTIONS
[2.5062191486358643]


  from ._conv import register_converters as _register_converters
2018-06-30 05:06:37.292868: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA

  from ._conv import register_converters as _register_converters
2018-06-30 05:06:37.292868: I tensorflow/core/platform/cpu_feature_guard.cc:140] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA



<h2> Running locally using gcloud </h2>

In [ ]:
%bash
rm -rf volume_estimator.tar.gz volume_estimator_trained
gcloud ml-engine local train \
   --module-name=trainer.task \
   --package-path=${PWD}/volume_estimator/trainer \
   -- \
   --train_steps=1000 \
   --output_dir=${PWD}/volume_estimator_trained 

When I ran it (due to random seeds, your results will be different), the ```average_loss``` (Mean Squared Error) on the evaluation dataset was 187, meaning that the RMSE was around 13.

In [ ]:
from google.datalab.ml import TensorBoard
TensorBoard().start('./volume_estimator_trained')

In [ ]:
for pid in TensorBoard.list()['pid']:
  TensorBoard().stop(pid)
  print 'Stopped TensorBoard with pid {}'.format(pid)

If the above step (to stop TensorBoard) appears stalled, just move on to the next step. You don't need to wait for it to return.

In [ ]:
!ls $PWD/volume_estimator_trained

<h2> Submit training job using gcloud </h2>

First copy the training data to the cloud.  Then, launch a training job.

After you submit the job, go to the cloud console (http://console.cloud.google.com) and select <b>Machine Learning | Jobs</b> to monitor progress.  

<b>Note:</b> Don't be concerned if the notebook stalls (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud.  Use the Cloud Console link (above) to monitor the job.

In [ ]:
%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/volume_estimator/smallinput/

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/volume_estimator/smallinput/volume_estimator_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/volume_estimator/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --output_dir=$OUTDIR \
   --train_steps=10000

Don't be concerned if the notebook appears stalled (with a blue progress bar) or returns with an error about being unable to refresh auth tokens. This is a long-lived Cloud job and work is going on in the cloud. 

<b>Use the Cloud Console link to monitor the job and do NOT proceed until the job is done.</b>

<h2> Deploy model </h2>

Find out the actual name of the subdirectory where the model is stored and use it to deploy the model.  Deploying model will take up to <b>5 minutes</b>.

In [ ]:
%bash
gsutil ls gs://${BUCKET}/volume_estimator/smallinput/volume_estimator_trained/export/exporter

In [ ]:
%bash
MODEL_NAME="volume_estimator"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/volume_estimator/smallinput/volume_estimator_trained/export/exporter | tail -1)
echo "Run these commands one-by-one (the very first time, you'll create a model and then create a version)"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

<h2> Prediction </h2>

In [ ]:
%bash
gcloud ml-engine predict --model=volume_estimator --version=v1 --json-instances=./test.json

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'r': 1.0,
        'h': 1.0
      }
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'volume_estimator', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print "response={0}".format(response)

<h2> Train on larger dataset </h2>

I have already followed the steps below and the files are already available. <b> You don't need to do the steps in this comment. </b> In the next chapter (on feature engineering), we will avoid all this manual processing by using Cloud Dataflow.

Go to http://bigquery.cloud.google.com/ and type the query:
<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  'nokeyindata' AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  AND ABS(HASH(pickup_datetime)) % 1000 == 1
</pre>

Note that this is now 1,000,000 rows (i.e. 100x the original dataset).  Export this to CSV using the following steps (Note that <b>I have already done this and made the resulting GCS data publicly available</b>, so you don't need to do it.):
<ol>
<li> Click on the "Save As Table" button and note down the name of the dataset and table.
<li> On the BigQuery console, find the newly exported table in the left-hand-side menu, and click on the name.
<li> Click on "Export Table"
<li> Supply your bucket name and give it the name train.csv (for example: gs://cloud-training-demos-ml/taxifare/ch3/train.csv). Note down what this is.  Wait for the job to finish (look at the "Job History" on the left-hand-side menu)
<li> In the query above, change the final "== 1" to "== 2" and export this to Cloud Storage as valid.csv (e.g.  gs://cloud-training-demos-ml/taxifare/ch3/valid.csv)
<li> Download the two files, remove the header line and upload it back to GCS.
</ol>

<p/>
<p/>

<h2> Run Cloud training on 1-million row dataset </h2>

This took 60 minutes and uses as input 1-million rows.  The model is exactly the same as above. The only changes are to the input (to use the larger dataset) and to the Cloud MLE tier (to use STANDARD_1 instead of BASIC -- STANDARD_1 is approximately 10x more powerful than BASIC).  At the end of the training the loss was 32, but the RMSE (calculated on the validation dataset) was stubbornly at 9.03. So, simply adding more data doesn't help.

In [17]:
%%bash

OUTDIR=gs://${BUCKET}/volume_estimator/ch3/volume_estimator_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
CRS_BUCKET=cloud-training-demos # use the already exported data
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/volume_estimator/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version=$TFVERSION \
   -- \
   --output_dir=$OUTDIR \
   --train_steps=100000 \
   --train_data_size=10000000 \
   --eval_data_size=1000000

gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained us-central1 lab3a_180630_052152
jobId: lab3a_180630_052152
state: QUEUED
gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained us-central1 lab3a_180630_052152
jobId: lab3a_180630_052152
state: QUEUED


Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/#1530336029011604...
Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/checkpoint#1530336031536858...
Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/eval/#1530335905039643...
Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/eval/events.out.tfevents.1530335905.cmle-training-master-a0a68e39e1-0-thrl8#1530336041286495...
Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/events.out.tfevents.1530335751.cmle-training-master-a0a68e39e1-0-thrl8#1530336034762482...
Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/export/#1530335908701638...
Removing gs://qwiklabs-gcp-04527c06d4f6cef3-bucket/volume_estimator/ch3/volume_estimator_trained/export/exporter/#15303359088

<h3>Deploy model</h3>

In [18]:
%bash
gsutil ls gs://${BUCKET}/volume_estimator/ch3/volume_estimator_trained/export/exporter

CommandException: One or more URLs matched no objects.
CommandException: One or more URLs matched no objects.


In [21]:
%bash
MODEL_NAME="volume_estimator"
MODEL_VERSION="v2"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/volume_estimator/ch3/volume_estimator_trained/export/exporter | tail -1)
echo "Run these commands one-by-one (the very first time, you'll create a model and then create a version)"
#gcloud ml-engine versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ml-engine models delete ${MODEL_NAME}
#gcloud ml-engine models create ${MODEL_NAME} --regions $REGION
gcloud ml-engine versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

Run these commands one-by-one (the very first time, you'll create a model and then create a version)
Run these commands one-by-one (the very first time, you'll create a model and then create a version)


Creating version (this might take a few minutes)......
............................................................................................done.
Creating version (this might take a few minutes)......
............................................................................................done.


<h3>Predict</h3>

In [ ]:
%bash
gcloud ml-engine predict --model=volume_estimator --version=v2 --json-instances=./test.json

## Challenge Exercise

Modify your solution to the challenge exercise in d_trainandevaluate.ipynb appropriately. Make sure that you implement training and deployment. Increase the size of your dataset by 10x since you are running on the cloud. Does your accuracy improve?

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License